In [87]:
import numpy as np
import pandas as pd

In [88]:
df = pd.read_csv('/kaggle/input/pgprnon-pgpr/pgpr_non_pgpr_clean.csv')

In [89]:
df.shape

(86446, 7)

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86446 entries, 0 to 86445
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         86446 non-null  int64  
 1   Orientation        86446 non-null  object 
 2   Protein accession  86446 non-null  object 
 3   Protein name       86446 non-null  object 
 4   Protein length     86446 non-null  float64
 5   DNA_sequence       86446 non-null  object 
 6   Target             86446 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 4.6+ MB


In [91]:
df.isnull().sum()

Unnamed: 0           0
Orientation          0
Protein accession    0
Protein name         0
Protein length       0
DNA_sequence         0
Target               0
dtype: int64

In [92]:
df = df.drop(columns=['Unnamed: 0', 'Protein accession', 'DNA_sequence'], axis= 1)

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86446 entries, 0 to 86445
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Orientation     86446 non-null  object 
 1   Protein name    86446 non-null  object 
 2   Protein length  86446 non-null  float64
 3   Target          86446 non-null  float64
dtypes: float64(2), object(2)
memory usage: 2.6+ MB


In [94]:
df['Protein name'].nunique()

14048

In [95]:
df.Target.unique()

array([1., 0.])

In [96]:
# Veri setindeki yinelenen gözlemleri kontrol eder ve bunları kaldırır

def duplicate_values(df):
    print("Duplicate check...")
    num_duplicates = df.duplicated(subset=None, keep='first').sum()
    if num_duplicates > 0:
        print("There are", num_duplicates, "duplicated observations in the dataset.")
        df.drop_duplicates(keep='first', inplace=True)
        print(num_duplicates, "duplicates were dropped!")
        print("No more duplicate rows!")
    else:
        print("There are no duplicated observations in the dataset.")

In [97]:
#duplicate_values(df)

In [98]:
X = df.drop('Target',axis=1)
y = df['Target']

In [99]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=11)

In [100]:
ohe = X_train.select_dtypes("object").columns
num = X_train.select_dtypes("float64").columns

In [101]:
num

Index(['Protein length'], dtype='object')

In [102]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler

ohe_enc = OneHotEncoder(handle_unknown='ignore') #OneHotEncoder
std_sca = StandardScaler()
column_trans = make_column_transformer((ohe_enc, ohe),(std_sca, num), remainder='passthrough')

In [103]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

operations = [("OneHotEncoder", column_trans),
              ("RF_model", RandomForestClassifier(random_state=11))]

pipe_model = Pipeline(steps=operations)

pipe_model.fit(X_train, y_train)

Pipeline(steps=[('OneHotEncoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Orientation', 'Protein name'], dtype='object')),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  Index(['Protein length'], dtype='object'))])),
                ('RF_model', RandomForestClassifier(random_state=11))])

In [107]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

In [108]:
def calculate_results(y_true, y_pred):
    model_accuracy = accuracy_score(y_true, y_pred)
    model_precision, model_recall, model_f1,_ = precision_recall_fscore_support(y_true, y_pred,average="weighted")
    model_results = {"accuracy":model_accuracy,
                     "precision":model_precision,
                     "recall" :model_recall,
                     "f1":model_f1}
    return model_results

In [109]:
calculate_results(y_true=y_test,
                  y_pred=(pipe_model.predict(X_test)))

{'accuracy': 0.6910931174089069,
 'precision': 0.6917918484995064,
 'recall': 0.6910931174089069,
 'f1': 0.6908134549492512}

In [111]:
column_trans.get_feature_names_out()

array(['onehotencoder__Orientation_minus',
       'onehotencoder__Orientation_plus',
       'onehotencoder__Protein name_(2,3-dihydroxybenzoyl)adenylate synthase',
       ...,
       'onehotencoder__Protein name_zincin-like metallopeptidase domain-containing protein',
       'onehotencoder__Protein name_zonular occludens toxin domain-containing protein',
       'standardscaler__Protein length'], dtype=object)

In [110]:
pipe_model["RF_model"].feature_importances_ 

array([8.61534571e-04, 6.77444891e-04, 3.25929779e-05, ...,
       5.49067620e-05, 1.99193182e-05, 3.20117965e-01])

In [112]:
features = pipe_model["OneHotEncoder"].get_feature_names_out()
features

array(['onehotencoder__Orientation_minus',
       'onehotencoder__Orientation_plus',
       'onehotencoder__Protein name_(2,3-dihydroxybenzoyl)adenylate synthase',
       ...,
       'onehotencoder__Protein name_zincin-like metallopeptidase domain-containing protein',
       'onehotencoder__Protein name_zonular occludens toxin domain-containing protein',
       'standardscaler__Protein length'], dtype=object)

In [114]:
new_features = [i.replace("onehotencoder__","").replace("remainder__", "") for i in features]
new_features

['Orientation_minus',
 'Orientation_plus',
 'Protein name_(2,3-dihydroxybenzoyl)adenylate synthase',
 'Protein name_(2,3-dihydroxybenzoyl)adenylate synthase EntE',
 'Protein name_(2E,6E)-farnesyl diphosphate synthase',
 'Protein name_(2E,6E)-farnesyl-diphosphate-specific ditrans,polycis-undecaprenyl-diphosphate synthase',
 'Protein name_(2Fe-2S)-binding protein',
 'Protein name_(3,5-dihydroxyphenyl)acetyl-CoA 1,2-dioxygenase DpgC',
 'Protein name_(3R)-hydroxymyristoyl-ACP dehydratase',
 'Protein name_(4S)-4-hydroxy-5-phosphonooxypentane-2,3-dione isomerase',
 'Protein name_(E)-4-hydroxy-3-methylbut-2-enyl-diphosphate synthase (flavodoxin)',
 'Protein name_(Fe-S)-binding protein',
 'Protein name_(Na+)-NQR maturation NqrM',
 'Protein name_(S)-acetoin forming diacetyl reductase',
 'Protein name_(S)-benzoin forming benzil reductase',
 'Protein name_(S)-ureidoglycine aminohydrolase',
 'Protein name_(d)CMP kinase',
 'Protein name_(dimethylallyl)adenosine tRNA methylthiotransferase',
 'Protei

In [116]:
df_fi = pd.DataFrame(data = pipe_model["RF_model"].feature_importances_, index=new_features, #index=X.columns
                      columns = ["Feature Importance"])
df_fi = df_fi.sort_values("Feature Importance", ascending=False)

df_fi.head(10)

,Feature Importance
standardscaler__Protein length,0.320118
Protein name_hypothetical protein,0.029448
Protein name_LysR family transcriptional regulator,0.003857
Protein name_acyl-CoA dehydrogenase family protein,0.001291
Protein name_helix-turn-helix domain-containing protein,0.001177
Protein name_MFS transporter,0.001154
Protein name_ABC transporter ATP-binding protein,0.001054
Protein name_TetR/AcrR family transcriptional regulator,0.001051
Protein name_cytochrome P450,0.000988
Protein name_ATP-binding protein,0.000941


In [82]:
from xgboost import XGBClassifier

In [83]:
operations = [("OneHotEncoder", column_trans),
              ("XGB_model", XGBClassifier(random_state=11))]

pipe_model = Pipeline(steps=operations)

pipe_model.fit(X_train, y_train)

Pipeline(steps=[('OneHotEncoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['Orientation', 'Protein name'], dtype='object')),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  Index(['Protein length'], dtype='object'))])),
                ('XGB_model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               co...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=11, ...))])

In [84]:
calculate_results(y_true=y_test,
                  y_pred=(pipe_model.predict(X_test)))

{'accuracy': 0.6045112781954888,
 'precision': 0.6570734568028934,
 'recall': 0.6045112781954888,
 'f1': 0.5684362475539015}

# ANN

In [29]:
X_train

,Orientation,Protein name,Protein length
46032,plus,glycoside hydrolase,733.0
82630,plus,hypothetical protein,57.0
6783,plus,hypothetical protein,31.0
83620,minus,OmpA family lipoprotein,219.0
86383,minus,hypothetical protein,66.0
...,...,...,...
23452,plus,hypothetical protein,87.0
75453,plus,DUF4158 domain-containing protein,344.0
40904,plus,YcgL domain-containing protein,86.0
47111,plus,crotonase/enoyl-CoA hydratase family protein,268.0


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86446 entries, 0 to 86445
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Orientation     86446 non-null  object 
 1   Protein name    86446 non-null  object 
 2   Protein length  86446 non-null  float64
 3   Target          86446 non-null  float64
dtypes: float64(2), object(2)
memory usage: 2.6+ MB


In [31]:
cat = X_train.select_dtypes("object").columns
cat
num = X_train.select_dtypes("float64").columns
num

Index(['Protein length'], dtype='object')

In [23]:
num

Index(['Protein length'], dtype='object')

In [38]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

enc = OneHotEncoder()
enc.fit_transform(X_train[cat])
X_test[cat] = enc.transform(X_test[cat])

ValueError: Found unknown categories ['DNA-binding transcriptional regulator/c-di-GMP phosphodiesterase PdeL', 'YmaF family protein', 'excinuclease ABC subunit A', 'uncharacterized protein YebO', 'putative transposase', 'abortive infection system antitoxin AbiGi family protein', 'propionyl-CoA:succinate CoA transferase', 'D-arabinose 5-phosphate isomerase GutQ', '(deoxy)nucleoside triphosphate pyrophosphohydrolase', 'DNA-binding transcriptional dual regulator AsnC', 'putative adenosine monophosphate--protein transferase Fic', 'phage DNA ejection protein', 'electron transfer flavoprotein subunit alpha', 'CRISPR-associated endoribonuclease Cas2', 'tetrahydrofolate dehydrogenase/cyclohydrolase catalytic domain-containing protein', 'V-type ATP synthase subunit E', 'type IV secretion protein Rhs', 'V-type ATP synthase subunit B', 'phage minor tail protein G', 'hydroxyacylglutathione hydrolase GloC', '2-oxoglutaramate amidase', 'DUF1440 domain-containing inner membrane protein YagU', 'thiol-

In [ ]:

sca = StandardScaler()
sca.fit_transform(X_train[num])
X_test[num] = sca.transform(X_test[num])

In [34]:
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from keras import callbacks

np.random.seed(0)

2024-06-03 18:53:12.440023: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-03 18:53:12.440199: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-03 18:53:12.650502: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [85]:
#Early stopping
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# Initialising the NN
model = Sequential()

# layers

model.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', input_dim = 26))
model.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
opt = Adam(learning_rate=0.00009)
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

# Train the ANN
history = model.fit(X_train, y_train, batch_size = 32, epochs = 150, callbacks=[early_stopping], validation_split=0.2)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: could not convert string to float: 'plus'

In [ ]:
history_df = pd.DataFrame(history.history)

plt.plot(history_df.loc[:, ['loss']], "#BDE2E2", label='Training loss')
plt.plot(history_df.loc[:, ['val_loss']],"#C2C4E2", label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")

plt.show()

In [ ]:
history_df = pd.DataFrame(history.history)

plt.plot(history_df.loc[:, ['accuracy']], "#BDE2E2", label='Training accuracy')
plt.plot(history_df.loc[:, ['val_accuracy']], "#C2C4E2", label='Validation accuracy')

plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()